In [5]:
import argparse

from utils import (print_yaml_config,init_or_resume_from, read_yaml)
from model_training.training_utils import load_for_inference
from constants import TOKENIZER_SEPECIAL_TOKENS
from training_datasets.dataset_utils import load_rm_dataset

def main(conf):
    print(f"\n{'==='*10} Following are the configuration for training{'==='*10}")
    print_yaml_config(conf)
    device_map = {"":0}
    special_tokens = TOKENIZER_SEPECIAL_TOKENS["llama"]
    model, tokenizer = load_for_inference(device_map,conf,special_tokens,conf.model_name,True)
    return model, tokenizer

config = {}
conf = read_yaml('./config.yaml')
config.update(conf["default"])
config.update(conf["eval_ranking_rm"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "eval_ranking_rm"

# Create a Namespace object for config
config_ranking = argparse.Namespace(**config)

config = {}
config.update(conf["default"])
config.update(conf["eval_abs_rm"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "eval_abs_rm"

# Create a Namespace object for config
config_abs = argparse.Namespace(**config)

_ , eval_abs = load_rm_dataset(config_abs)

_ , eval_ranking = load_rm_dataset(config_ranking)


In [6]:
config_ranking.adapter_name = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
config_ranking.init_from_adapter = "LLama-2-7b_ours_oasst_sft_reward_ranking_bs16__qlora_default"
init_or_resume_from(config_ranking)
config_ranking.model_name = config_ranking.merged_adapter_path if config_ranking.merged_adapter_path else config_ranking.base_model_name
ranking_reward_model, ranking_reward_tokenizer = main(config_ranking)

config_abs.adapter_name = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
config_abs.init_from_adapter = "LLama-2-7b_ours_oasst_sft_reward_abs_bs64_qlora_default"
init_or_resume_from(config_abs)
config_abs.model_name = config_abs.merged_adapter_path if config_abs.merged_adapter_path else config_abs.base_model_name
abs_reward_model, abs_reward_tokenizer = main(config_abs)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_reward_ranking_bs16__qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: null
adam_beta2: 0.999
adam_epsilon: null
adapter_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adapter_number: final_checkpoint
adpater_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adpater_path: output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint
base_model_name: meta-llama/Llama-2-7b-hf
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  oasst_export:
    lang: en,es,de,fr
    val_split: 0.05
debug: false
debug_set: 100
dtype: bf16
early_sto

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.47s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/merged and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==============================Now Loading Peft model output/LLama-2-7b_ours_oasst_sft_reward_ranking_bs16__qlora_default/final_checkpoint.
tokenizer size 32003
Resizing embeddings to 32016
============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_reward_abs_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: null
adam_beta2: 0.999
adam_epsilon: null
adapter_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adapter_number: final_checkpoint
adpater_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adpater_path: output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint
base_model_name: meta-llama/Llama-2-7b-hf
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  sam

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.31s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/merged and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==============================Now Loading Peft model output/LLama-2-7b_ours_oasst_sft_reward_abs_bs64_qlora_default/final_checkpoint.
tokenizer size 32003
Resizing embeddings to 32016


In [7]:
from training_datasets.collators import AbsoluteScoreDataCollator, RankingDataCollator


abs_collate_fn = AbsoluteScoreDataCollator(
    abs_reward_tokenizer,
    max_length=config_abs.collator["max_length"],
    pad_to_multiple_of=16,
)

ranking_collate_fn = RankingDataCollator(
    ranking_reward_tokenizer,
    max_length=config_ranking.collator["max_length"],
    pad_to_multiple_of=16,
    max_replies=config_ranking.max_replies
)

# eval_abs,eval_ranking

In [48]:
inputs, cu_lens = ranking_collate_fn([eval_ranking["oasst_export"][90]])
inputs.input_ids.shape



torch.Size([3, 352])

In [58]:
idx = 30

abs = []
rr = []

for idx in range(len(eval_abs["oasst_export_abs"])):
        data = eval_abs["oasst_export_abs"][idx]
        inputs = abs_collate_fn([data])
        labels = inputs.pop("labels")
        print('***'*10)
        print(f'label:{labels}')


        logits = abs_reward_model(input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                use_cache=False,
                ).logits
        abs.append(logits[0].item())
        print(logits[0].item())
        print('==='*10)
        logits = ranking_reward_model(input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                use_cache=False,
                ).logits
        rr.append(logits[0].item())
        print(logits[0].item())
        print('==='*10)

# print(abs_reward_tokenizer.batch_decode(inputs["input_ids"]))        


/home/alikhan/miniconda3/envs/torch_p310_setup/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


******************************
label:tensor([0.6667], dtype=torch.float64)
0.640625
0.28125
******************************
label:tensor([0.9167], dtype=torch.float64)
0.859375
1.859375
******************************
label:tensor([0.5833], dtype=torch.float64)
0.82421875
0.484375
******************************
label:tensor([0.5833], dtype=torch.float64)
0.69921875
0.58984375
******************************
label:tensor([0.5000], dtype=torch.float64)
0.490234375
-2.015625
******************************
label:tensor([0.8333], dtype=torch.float64)
0.73046875
1.375
******************************
label:tensor([0.7500], dtype=torch.float64)
0.765625
0.85546875
******************************
label:tensor([0.8333], dtype=torch.float64)
0.828125
1.359375
******************************
label:tensor([0.5833], dtype=torch.float64)
0.46875
-0.349609375
******************************
label:tensor([0.9167], dtype=torch.float64)
0.953125
1.3046875
******************************
label:tensor([0.9167], dt

In [62]:
min(rr)

-2.859375

## SFT


In [ ]:
import argparse

from utils import (print_yaml_config,init_or_resume_from, read_yaml)
from model_training.training_utils import load_for_inference
from constants import TOKENIZER_SEPECIAL_TOKENS
from training_datasets.dataset_utils import load_sft_dataset, load_rm_dataset
from training_datasets.collators import DialogueDataCollator




def main(conf):
    print(f"\n{'==='*10} Following are the configuration for training{'==='*10}")
    print_yaml_config(conf)
    device_map = {"":0}
    special_tokens = TOKENIZER_SEPECIAL_TOKENS["llama"]
    model, tokenizer = load_for_inference(device_map,conf,special_tokens,conf.model_name,False)
    return model, tokenizer

config = {}
conf = read_yaml('./config.yaml')
config.update(conf["default"])
config.update(conf["sft_eval"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "sft_eval"

# Create a Namespace object for config
config_ns = argparse.Namespace(**config)

init_or_resume_from(config_ns)

In [5]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
config_ns.adapter_number= "final_checkpoint"
init_or_resume_from(config_ns)
ep2_qlora_sft_model, ep2_qlora_sft_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_s

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.80s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [4]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default"
config_ns.adapter_number= "final_checkpoint"
init_or_resume_from(config_ns)
ep3_qlora_sft_model, ep3_qlora_sft_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_s

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [11]:
m_t = {
       # "LLama-2-7b_full_sft_4bit_bs_64":[full_sft_model,full_sft_tokenizer,0.8,0.9,50],
       "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default":[ep2_qlora_sft_model, ep2_qlora_sft_tokenizer,0.8,0.9,40],
       # "LLama-2-7b_ours_oasst_sft_only_bs64_qlora_default": [only_oasst_model, only_oasst_tokenizer,0.85,0.9,40],
       "LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default":[ep3_qlora_sft_model, ep3_qlora_sft_tokenizer,0.8,0.9,0]
}

In [7]:
def get_pred(m_t,input):
    for key,value in m_t.items():
        model,tokenizer,temp,top_p,top_k = value
        tokens = tokenizer(input,return_tensors='pt')
        output = model.generate(**tokens,
                        max_new_tokens=512,
                        do_sample=True,
                        top_p=top_p,
                        top_k=top_k,
                        repetition_penalty=1.2,
                        temperature=temp,
                    )
        print(f"\n\nOutput for model {key}:\n{'-'*100}")
        print(tokenizer.decode(output[0], skip_special_tokens=False))




In [21]:

input = """<|prompter|>Considering the advancements in quantum computing and its potential implications for classical encryption methods, explain how current cybersecurity measures might need to adapt, and what challenges or benefits this new computational paradigm could present to the fields of data privacy and digital communication.</s><|assistant|>"""
get_pred(m_t,input)




Output for model LLama-2-7b_full_sft_4bit_bs_64:
----------------------------------------------------------------------------------------------------
<s><|prompter|> Considering the advancements in quantum computing and its potential implications for classical encryption methods, explain how current cybersecurity measures might need to adapt, and what challenges or benefits this new computational paradigm could present to the fields of data privacy and digital communication.</s><|assistant|> With the advent of quantum computers and their ability to break certain types of cryptographic algorithms used in traditional security measures such as RSA and elliptic curve, it is important that existing systems are updated with newer, more secure ones which can be protected against these attacks.
The main challenge faced by organizations will likely include creating a balance between providing adequate levels of protection while also keeping up with rapidly changing technologies.  Additionally